In [3]:
from tensorflow.keras.datasets import imdb 

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [153]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [182]:
#TODO: Please download this file from
# https://www.kaggle.com/datasets/mzohaibzeeshan/thyroid-cancer-risk-dataset?resource=download


In [19]:
data = pd.read_csv('thyroid_cancer_risk_data.csv')

In [48]:
y_df = data[["Thyroid_Cancer_Risk"]]

In [57]:
data["Thyroid_Cancer_Risk"].value_counts()

Low       108388
Medium     72400
High       31903
Name: Thyroid_Cancer_Risk, dtype: int64

In [170]:
name = "Thyroid_Cancer_Risk"
y_df["risk"] = data[name].map({'Low':0,'Medium':1, 'High':2})

/usr/local/anaconda/envs/tensorflow2/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [171]:
y = y_df["risk"].to_numpy()

In [173]:
y

array([0, 0, 0, ..., 1, 1, 0])

In [122]:
numeric_features = ["Age","TSH_Level","T3_Level","T4_Level","Nodule_Size"]
categorical_features = ["Gender","Country","Ethnicity","Family_History","Radiation_Exposure","Iodine_Deficiency","Smoking","Obesity","Diabetes"]
features = numeric_features + categorical_features

In [158]:
features

['Age',
 'TSH_Level',
 'T3_Level',
 'T4_Level',
 'Nodule_Size',
 'Gender',
 'Country',
 'Ethnicity',
 'Family_History',
 'Radiation_Exposure',
 'Iodine_Deficiency',
 'Smoking',
 'Obesity',
 'Diabetes']

In [159]:
for f in categorical_features:
    print(data[f].value_counts())

Female    127527
Male       85164
Name: Gender, dtype: int64
India          42496
China          31978
Nigeria        31918
Brazil         21413
Russia         21297
Japan          16867
South Korea    14965
UK             10642
USA            10558
Germany        10557
Name: Country, dtype: int64
Caucasian         63669
Asian             53261
African           42414
Hispanic          32012
Middle Eastern    21335
Name: Ethnicity, dtype: int64
No     148866
Yes     63825
Name: Family_History, dtype: int64
No     180831
Yes     31860
Name: Radiation_Exposure, dtype: int64
No     159673
Yes     53018
Name: Iodine_Deficiency, dtype: int64
No     170260
Yes     42431
Name: Smoking, dtype: int64
No     148805
Yes     63886
Name: Obesity, dtype: int64
No     170098
Yes     42593
Name: Diabetes, dtype: int64


In [125]:
X_df = data[features]

In [154]:

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), 
           ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
#         ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [155]:
X = preprocessor.fit_transform(X_df)

In [156]:
X.shape, y.shape

((212691, 34), (212691,))

In [136]:
from sklearn.model_selection import train_test_split

In [175]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [176]:
X_train.shape

(170152, 34)

In [177]:
y_train.shape

(170152,)

In [178]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(3, activation="softmax")
])

In [179]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 30)                1050      
_________________________________________________________________
dense_3 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 93        
Total params: 2,073
Trainable params: 2,073
Non-trainable params: 0
_________________________________________________________________


In [180]:
sgd = keras.optimizers.SGD(learning_rate=0.01)
# Compiling the model we created

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=sgd,
              metrics=["accuracy"])

In [181]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_test, y_test)) # fitting the model to training data

Epoch 1/30
5318/5318 [==============================] - 10s 2ms/step - loss: 0.7813 - accuracy: 0.5879 - val_loss: 0.6414 - val_accuracy: 0.6455
Epoch 2/30
5318/5318 [==============================] - 10s 2ms/step - loss: 0.5979 - accuracy: 0.6571 - val_loss: 0.5786 - val_accuracy: 0.6606
Epoch 3/30
5318/5318 [==============================] - 12s 2ms/step - loss: 0.5774 - accuracy: 0.6592 - val_loss: 0.5748 - val_accuracy: 0.6606
Epoch 4/30
5318/5318 [==============================] - 13s 2ms/step - loss: 0.5752 - accuracy: 0.6592 - val_loss: 0.5732 - val_accuracy: 0.6606
Epoch 5/30
5318/5318 [==============================] - 9s 2ms/step - loss: 0.5745 - accuracy: 0.6593 - val_loss: 0.5744 - val_accuracy: 0.6606
Epoch 6/30
5318/5318 [==============================] - 9s 2ms/step - loss: 0.5741 - accuracy: 0.6594 - val_loss: 0.5726 - val_accuracy: 0.6606
Epoch 7/30
5318/5318 [==============================] - 9s 2ms/step - loss: 0.5739 - accuracy: 0.6594 - val_loss: 0.5748 - val_accur

KeyboardInterrupt: 

In [184]:
# 1. If you have an image, Flatten
# 2. If your columns are numeric, you need to fill missing value and scale
# 3. if your columns are categorical, you would need to OneHotEncoding
# 4. If your data has text, convert it into embedding or older method tfidf